# Technical Factor - Simple Moving Average (SMA)

In [80]:
import dai
import pandas as pd

In [81]:
sd = '2025-01-01'
ed = '2026-01-01'

## Definition

**SMA (Simple Moving Average)** is one of the most fundamental technical indicators in financial markets. It represents the **average price over a fixed number of past periods**, helping to smooth short-term fluctuations and reveal the underlying market direction.

The SMA is calculated as the arithmetic mean of prices over the past **N periods** (most commonly closing prices):

$$
SMA_M(t)=\frac{1}{N}\sum_{i=0}^{N-1} Close(t-i)
$$

Where:
- $N$: the lookback period  
- $Close(t-i)$: the price at time $t-i$

In [82]:
def get_sql_factor(sd, ed, ns, nm, nl, data_base):

    sql_factor = f"""--sql
    SELECT
        date,
        instrument,
        close,
        m_ta_sma(close, {ns}) AS SMA_S,
        m_ta_sma(close, {nm}) AS SMA_M,
        m_ta_sma(close, {nl}) AS SMA_L,
    FROM {data_base}
    """

    return sql_factor

## Signals

### Trend Signals

Trend signals describe the **primary market direction** and are typically derived from SMA crossovers or the slope of longer-term averages.

- **Trend Cross**

    - **Gold Cross:** The short-term SMA crosses above the long-term SMA

    $$
    SMA_S(t-1) \le SMA_L(t-1)
    \quad \land \quad
    SMA_S(t) > SMA_L(t)
    $$

    - **Dead Cross:** The short-term SMA crosses below the long-term SMA

    $$
    SMA_S(t-1) \le SMA_L(t-1)
    \quad \land \quad
    SMA_S(t) > SMA_L(t)
    $$

- **Trend Confirmation**

    - **Upward Trend:** Current price is above a rising long-term SMA.

    $$
    Close_t > SMA_L(t)
    \quad \land \quad
    SMA_L(t) - SMA_L(t-1) > 0
    $$

    - **Downward Trend:** Current price is below a declining long-term SMA.

    $$
    Close_t < SMA_l(t)
    \quad \land \quad
    SMA_l(t) - SMA_l(t-1) < 0
    $$

In [83]:
def get_sql_signal_trend(sd, ed, ns, nm, nl, data_base):

    sql_signal_trend = f"""
    SELECT
        date,
        instrument,
        -- Trend Cross
        IF(SMA_S > SMA_L AND m_lag(SMA_S, 1) < m_lag(SMA_L,1), 1, 0) AS TRBY1,
        IF(SMA_S < SMA_L AND m_lag(SMA_S, 1) > m_lag(SMA_L,1), 1, 0) AS TRSL1,
        -- Trend Confirmation
        IF(close > SMA_L AND SMA_L > m_lag(SMA_L, 1), 1, 0) AS TRBY2,
        IF(close < SMA_L AND SMA_L < m_lag(SMA_L, 1), 1, 0) AS TRSL2,
    FROM {data_base}
    """

    return sql_signal_trend

---

### Momentum Signals

Momentum signals describe the **strength and acceleration of price movement**.  
Unlike trend signals, which focus on direction, momentum signals focus on **how fast the trend is changing**, commonly derived from the slope of SMAs or the expansion/contraction between SMAs.

- **SMA Slope**

    - **Momentum Up:** The short-term SMA is rising and accelerating upward.

    $$
    SMA_S(t) - SMA_S(t-1) > 0
    \quad \land \quad
    [SMA_S(t) - SMA_S(t-1)] > [SMA_S(t-1) - SMA_S(t-2)]
    $$

    - **Momentum Down:** The short-term SMA is falling and accelerating downward.

    $$
    SMA_S(t) - SMA_S(t-1) < 0
    \quad \land \quad
    [SMA_S(t) - SMA_S(t-1)] < [SMA_S(t-1) - SMA_S(t-2)]
    $$

- **SMA Spread Expansion**

    - **Bullish Momentum:** The distance between short- and long-term SMAs is positive and widening.

    $$
    (SMA_S - SMA_L)_t > 0
    \quad \land \quad
    (SMA_S - SMA_L)_t - (SMA_S - SMA_L)_{t-1} > 0
    $$

    - **Bearish Momentum:** The SMA spread is negative and widening to the downside.

    $$
    (SMA_S - SMA_L)_t < 0
    \quad \land \quad
    (SMA_S - SMA_L)_t - (SMA_S - SMA_L)_{t-1} < 0
    $$

In [84]:
def get_sql_signal_momentum(sd, ed, ns, nm, nl, data_base):

    sql_signal_momentum = f"""--sql
    SELECT
        date,
        instrument,
        -- SMA Slope
        IF(SMA_S - m_lag(SMA_S, 1) > 0 AND (SMA_S - m_lag(SMA_S, 1)) > (m_lag(SMA_S, 1) - m_lag(SMA_S, 2)), 1, 0) AS MTBY1,
        IF(SMA_S - m_lag(SMA_S, 1) < 0 AND (SMA_S - m_lag(SMA_S, 1)) < (m_lag(SMA_S, 1) - m_lag(SMA_S, 2)), 1, 0) AS MTSL1,
        -- SMA Spread Expansion
        IF((SMA_S - SMA_L) > 0 AND ((SMA_S - SMA_L) - (m_lag(SMA_S, 1) - m_lag(SMA_L, 1))) > 0, 1, 0) AS MTBY2,
        IF((SMA_S - SMA_L) < 0 AND ((SMA_S - SMA_L) - (m_lag(SMA_S, 1) - m_lag(SMA_L, 1))) < 0, 1, 0) AS MTSL2,
    FROM {data_base}
    """
    
    return sql_signal_momentum


---

### Reversal Signals

Reversal signals identify **mean-reversion behavior**, where price deviates from its average and begins to move back toward it.

- **Price-Based Reversal**

    - **Reverse Oversold:** Price is below the SMA and starts to move upward.

    $$
    Close_t < SMA_M(t)
    \quad \land \quad
    Close_t > Close_{t-1}
    $$

    - **Reverse Overbought:** Price is above the SMA and starts to move downward.

    $$
    Close_t > SMA_M(t)
    \quad \land \quad
    Close_t < Close_{t-1}
    $$

- **Deviation-Based Reversal (Z-score)**

    - **Reverse Oversold:** Price is statistically oversold and deviation starts to contract.

    $$
    Z_t < -z_0
    \quad \land \quad
    Z_t - Z_{t-1} > 0
    $$

    - **Reverse Overbought:** Price is statistically overbought and deviation starts to contract.

    $$
    Z_t > z_0
    \quad \land \quad
    Z_t - Z_{t-1} < 0
    $$

    - Where:

    $$Z_t = \frac{Close_t - SMA_M(t)}{\sigma_M(t)}$$
    $$\sigma_M(t) = \sqrt{\frac{1}{N}\sum_{i=0}^{N-1}\left(Close_{t-i} - SMA_M(t)\right)^2}$$

In [85]:
def get_sql_signal_reversal(sd, ed, ns, nm, nl, data_base):

    sql_signal_reversal = f"""--sql
    SELECT
        date,
        instrument,
        (close - SMA_M) / m_stddev(close, {nm}) AS Z,
        -- Price Based Reversal
        IF(close < SMA_M AND close > m_lag(close, 1), 1, 0) AS RVBY1,
        IF(close > SMA_M AND close < m_lag(close, 1), 1, 0) AS RVSL1,
        -- Deviation Basef Reversal (z0 set as 2)
        IF(Z < -2 AND (Z - m_lag(Z, 1)) > 0, 1, 0) AS RVBY2,
        IF(Z > +2 AND (Z - m_lag(Z, 1)) < 0, 1, 0) AS RVSL2,
    FROM {data_base}
    """
    
    return sql_signal_reversal


---

### Breakout Signals

Breakout signals identify moments when price **decisively moves beyond an SMA-based support or resistance level**, often marking the beginning of a new expansion phase.

- **Band-Based Breakout**

    - **Breaking Resistance:** Price breaks above the SMA upper band.

    $$
    Close_{t-1} \le SMA_M(t-1) + k \cdot \sigma_M(t-1)
    \quad \land \quad
    Close_t > SMA_M(t) + k \cdot \sigma_M(t)
    $$

    - **Breaking Support:** Price breaks below the SMA lower band.

    $$
    Close_{t-1} \ge SMA_M(t-1) - k \cdot \sigma_M(t-1)
    \quad \land \quad
    Close_t < SMA_M(t) - k \cdot \sigma_M(t)
    $$

- **Major SMA Level Break**

    - **Breaking Resistance:** Price crosses above a major long-term SMA.

    $$
    Close_{t-1} \le SMA_L(t-1)
    \quad \land \quad
    Close_t > SMA_L(t)
    $$

    - **Breaking Support:** Price crosses below a major long-term SMA.

    $$
    Close_{t-1} \ge SMA_L(t-1)
    \quad \land \quad
    Close_t < SMA_L(t)
    $$

In [86]:
def get_sql_signal_breakout(sd, ed, ns, nm, nl, data_base):

    sql_signal_breakout = f"""
    SELECT
        date,
        instrument,
        m_stddev(close, {nm}) AS sigma_M,
        -- Band Based Breakout (k set as 2)
        IF(m_lag(close, 1) <= (m_lag(SMA_M, 1) + 2 * m_lag(sigma_M, 1)) AND close > (SMA_M + 2 * sigma_M), 1, 0) AS BKBY1,
        IF(m_lag(close, 1) >= (m_lag(SMA_M, 1) - 2 * m_lag(sigma_M, 1)) AND close < (SMA_M - 2 * sigma_M), 1, 0) AS BKSL1,
        -- Major SMA Level Break
        IF(m_lag(close, 1) <= m_lag(SMA_L, 1) AND close > SMA_L, 1, 0) AS BKBY2,
        IF(m_lag(close, 1) >= m_lag(SMA_L, 1) AND close < SMA_L, 1, 0) AS BKSL2,
    FROM {data_base}
    """
    
    return sql_signal_breakout


## Combined Factor Data

In [87]:
def alpha_ta_sma(sd, ed, ns, nm, nl, data_base):

    sql = f"""--sql
    WITH
    data_base AS (
        {get_sql_factor(sd, ed, ns, nm, nl, data_base)}
    ),
    data_signal_trend AS (
        {get_sql_signal_trend(sd=sd, ed=ed, ns=ns, nm=nm, nl=nl, data_base='data_base')}
    ),
    data_signal_momentum AS (
        {get_sql_signal_momentum(sd=sd, ed=ed, ns=ns, nm=nm, nl=nl, data_base='data_base')}
    ),
    data_signal_reversal AS (
        {get_sql_signal_reversal(sd=sd, ed=ed, ns=ns, nm=nm, nl=nl, data_base='data_base')}
    ),
    data_signal_breakout AS (
        {get_sql_signal_breakout(sd=sd, ed=ed, ns=ns, nm=nm, nl=nl, data_base='data_base')}
    ),
    data_combined AS (
        SELECT
            date,
            instrument,
            SMA_S,
            SMA_M,
            SMA_L,
            TRBY1,
            TRSL1,
            TRBY2,
            TRSL2,
            MTBY1,
            MTSL1,
            MTBY2,
            MTSL2,
            RVBY1,
            RVSL1,
            RVBY2,
            RVSL2,
            RVBY1,
            RVSL1,
            RVBY2,
            RVSL2,
            BKBY1,
            BKSL1,
            BKBY2,
            BKSL2,
        FROM data_base 
        JOIN data_signal_trend    USING (date, instrument)
        JOIN data_signal_momentum USING (date, instrument)
        JOIN data_signal_reversal USING (date, instrument)
        JOIN data_signal_breakout USING (date, instrument)
    ) 
    SELECT *
    FROM data_combined
    ORDER BY date, instrument
    """

    return dai.query(sql, filters={'date':[sd, ed]}).df().dropna()

In [88]:
alpha_ta_sma(sd=sd, ed=ed, ns=5, nm=10, nl=20, data_base='cn_stock_bar1d')

,date,instrument,SMA_S,SMA_M,SMA_L,TRBY1,TRSL1,TRBY2,TRSL2,MTBY1,...,RVBY2,RVSL2,RVBY1_1,RVSL1_1,RVBY2_1,RVSL2_1,BKBY1,BKSL1,BKBY2,BKSL2
102396,2025-02-06,000001.SZ,1453.164766,1453.164766,1455.081533,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
102397,2025-02-06,000002.SZ,1308.630639,1285.554259,1272.108179,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
102398,2025-02-06,000004.SZ,44.596820,47.132670,50.158215,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
102399,2025-02-06,000006.SZ,272.455202,278.336674,275.395938,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
102400,2025-02-06,000007.SZ,57.669924,58.216637,57.645074,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1306369,2025-12-29,920978.BJ,47.184848,46.218368,45.966439,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1306370,2025-12-29,920981.BJ,46.854676,47.134359,46.836748,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1306371,2025-12-29,920982.BJ,1069.369897,1075.601856,1053.978457,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
1306372,2025-12-29,920985.BJ,12.278648,12.260739,12.745080,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
